In [15]:
# 导入及处理数据
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import date

In [16]:
security = 'META'
# 绘制数据的规模
scale = 500
# df = import_csv(security)[-scale:]
# df = pd.read_csv("Data/{}.csv".format(security), index_col=0, parse_dates=True)
df = yf.download(security)
df = yf.download(security, start='2023-01-04', end=date.today(), interval='1d')
df = df.tail(200)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [17]:
# SMA:简单移动平均(Simple Moving Average)
time_period = 30  # SMA的计算周期，默认为20
stdev_factor = 2  # 上下频带的标准偏差比例因子
history = []  # 每个计算周期所需的价格数据
sma_values = []  # 初始化SMA值
upper_band = []  # 初始化阻力线价格
lower_band = []  # 初始化支撑线价格

In [18]:
# 构造列表形式的绘图数据
for close_price in df['Close']:
    # 
    history.append(close_price)

    # 计算移动平均时先确保时间周期不大于20
    if len(history) > time_period:
        del (history[0])

    # 将计算的SMA值存入列表
    sma = np.mean(history)
    sma_values.append(sma)  
    # 计算标准差
    stdev = np.sqrt(np.sum((history - sma) ** 2) / len(history))  
    upper_band.append(sma + stdev_factor * stdev)
    lower_band.append(sma - stdev_factor * stdev)

In [19]:
# 将计算的数据合并到DataFrame
df = df.assign(Close_Price=pd.Series(df['Close'], index=df.index))
df = df.assign(moving_average=pd.Series(sma_values, index=df.index))
df = df.assign(upper_band=pd.Series(upper_band, index=df.index))
df = df.assign(lower_band=pd.Series(lower_band, index=df.index))

In [21]:
# 绘图
ax = plt.figure(figsize=(16,8), dpi=1200)
plt.title('{} Bollinger Band'.format(security), fontsize=20)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
# 设定y轴标签
ax.ylabel = '%s price' % (security)

df['Close_Price'].plot(color='k', lw=1., legend=True)
df['moving_average'].plot(color='b', lw=1., legend=True)
df['upper_band'].plot(color='r', lw=1., legend=True)
df['lower_band'].plot(color='g', lw=1., legend=True)
plt.show()